from _ast import Load

# Simple notebook to process generic webaccess files. 

Assumptions
1. There are one or more csv files with access logs in the given directory. 
1. Each file can be big - 20k or more rows to process

Approach: 
1. Load each csv in the named directory with the given file spec (glob style)
1. combine as we go into a master df.     If the size becomes too large . then process each file one at a time and collect the stats as we go. 



In [48]:
import pandas as pd
import logging
from datetime import date, timedelta
import numpy as np
from datetime import date, datetime
import seaborn as sns
import os
import glob
import pytz
from df_util import check_df

logging.basicConfig(level=logging.DEBUG, format="%(message)s")
LOGGER = logging.getLogger(__name__)
LOGGER.debug("log level = %s", str(LOGGER.getEffectiveLevel()))


log level = 10


In [49]:
#files to load: 
if False:
    FILE_PATTERN="../data/webaccess/web_access_*.csv"
    #Columns that must be in the CSV. 
    REQUIRED_COLUMNS=['date','user','url']
else:

    FILE_PATTERN="../../../sample_webaccess_data/*UTC_web_log*csv"
    REQUIRED_COLUMNS=['Event Time', 'User', 'URL', 'Cloud Application Class', 'Cloud Application', 
                  'URL Class','URL Super Category', 'URL Category', 'Threat Category', 'Threat Name',
                  'Location', 'Department', 
                  #'Protocol', 'Blocked Policy Type',
                  'Upload File Name']
    


REQUIRED_VALUES=REQUIRED_COLUMNS

DATE_AS_OF=date.fromisoformat('2023-06-01') #Date for analysis

#For data visualization - date range sizes
DATE_GROUP_DAYS=30
CUTOFF_DAYS=90


In [50]:
#  Determine which  files that meet the filespec - this doesn't load them
file_names = glob.glob(FILE_PATTERN)
if len(file_names) == 0:
    msg = f"ERROR: No file names found for pattern (\"{FILE_PATTERN}\". Stopping"
    LOGGER.error(msg)
    raise ValueError(msg)

LOGGER.info("%d files met the file pattern:[%s]", len(file_names), FILE_PATTERN)
print(file_names)
    

1 files met the file pattern:[../../../sample_webaccess_data/*UTC_web_log*csv]


['../../../sample_webaccess_data/2024-03-24T20-49-01_UTC_web_log_cleaned.csv']


In [51]:
df = None

#TODO - skip any non value rows from top of CSV. 
for i, file_name in enumerate(file_names):
    df_tmp = pd.read_csv(file_name)
    LOGGER.debug("file %d:[%s]: Loaded %d rows", i, file_name, len(df_tmp))

    if df is None:
        df = df_tmp
    else:
        df = pd.concat([df, df_tmp], axis=0)
        LOGGER.debug(" after concat: df len= %d rows", len(df))
df.head()

file 0:[../../../sample_webaccess_data/2024-03-24T20-49-01_UTC_web_log_cleaned.csv]: Loaded 1000 rows


,No.,Event Time,User,URL,Policy Action,Cloud Application Class,Cloud Application,URL Class,URL Super Category,URL Category,...,DLP MD5,DLP Identifier,ALPN Protocol,Unscannable Type,Forwarding Rule,Forwarding Method,Application Segment,Gateway IP,Gateway Name,Upload File Name
0,130,"March 22, 2024 9:00:04 PM PDT",AAAAA,d.la1-core1.sfdc-lywfpd.salesforceliveagent.co...,Allowed,Productivity and CRM Tools,Salesforce,Business Use,Business and Economy,Professional Services,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,131,"March 22, 2024 9:00:04 PM PDT",AAAAA,d.la1-core1.sfdc-lywfpd.salesforceliveagent.co...,Allowed,Productivity and CRM Tools,Salesforce,Business Use,Business and Economy,Professional Services,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,151,"March 22, 2024 9:00:05 PM PDT",AAAAA,chat.google.com:443,Allowed,Instant Messaging,Google Chat,Business Use,Internet Communication,Online Chat,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,152,"March 22, 2024 9:00:05 PM PDT",AAAAA,chat.google.com/u/0/_/dynamitewebui/idv/,Allowed,Instant Messaging,Google Chat,Business Use,Internet Communication,Online Chat,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,368,"March 22, 2024 9:00:12 PM PDT",AAAAA,d.la1-core2.sfdc-lywfpd.salesforceliveagent.co...,Allowed,Productivity and CRM Tools,Salesforce,Business Use,Business and Economy,Professional Services,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:


#check dataframe for missing blocks.  err is empty if no errors - otherwise an err str
err = check_df(df, required_columns=REQUIRED_COLUMNS, required_values=REQUIRED_VALUES)
if err:
    print(str(err))
    Exception(err)
    
LOGGER.info("Loaded %d clean rows", len(df))

Num empty required cells in dataframe: 3000
Loaded 1000 clean rows


Data error. Dataframe contained 3000 empty cells in required columns ['Event Time', 'User', 'URL', 'Cloud Application Class', 'Cloud Application', 'URL Class', 'URL Super Category', 'URL Category', 'Threat Category', 'Threat Name', 'Location', 'Department', 'Upload File Name']. The returned list shows the row index and column names of the FIRST ERROR IN ROWS with blanks.  {0: 'Upload File Name', 1: 'Upload File Name', 2: 'Upload File Name', 3: 'Upload File Name', 4: 'Upload File Name', 5: 'Upload File Name', 6: 'Upload File Name', 7: 'Upload File Name', 8: 'Upload File Name', 9: 'Upload File Name', 10: 'Upload File Name', 11: 'Upload File Name', 12: 'Upload File Name', 13: 'Upload File Name', 14: 'Upload File Name', 15: 'Upload File Name', 16: 'Upload File Name', 17: 'Upload File Name', 18: 'Upload File Name', 19: 'Upload File Name', 20: 'Upload File Name', 21: 'Upload File Name', 22: 'Upload File Name', 23: 'Upload File Name', 24: 'Upload File Name', 25: 'Upload File Name', 26: 'Uploa

In [53]:
df.head()

# Assuming date_string is your datetime string
date_string = "March 22, 2024 9:00:25 PM PDT"

# Replace PDT with UTC-7
date_string = date_string.replace("PDT", "UTC-7")

# Use pandas to parse the string into a datetime
dt = pd.to_datetime(date_string)

# Print the datetime
print(dt)

dt_utc = dt.astimezone(pytz.UTC)
print(dt_utc)



2024-03-22 21:00:25+07:00
2024-03-22 14:00:25+00:00


In [54]:
# Parse the string into a datetime object
#  Assume the date is in a format like this: "March 22, 2024 9:00:25 PM PDT"
def convert_date(s:str) ->str:     
    # Replace PDT with UTC-7
    s = s.replace("PDT", "UTC-7")
    dt =  pd.to_datetime(s)
    # Convert the datetime object to UTC
    dt_utc = dt.astimezone(pytz.UTC)
    return dt_utc

df['datetime'] = df['Event Time'].apply(convert_date)
df['date']  = pd.to_datetime(df['datetime'], format='%Y-%m-%d').dt.date


LOGGER.info("Date-range min: %s", df['datetime'].min())
LOGGER.info("Date-range max: %s", df['datetime'].max())

LOGGER.info("Date-range min: %s", df['date'].min())
LOGGER.info("Date-range max: %s", df['date'].max())

df.head()


Date-range min: 2024-01-22 14:00:11+00:00
Date-range max: 2024-03-26 14:00:26+00:00
Date-range min: 2024-01-22
Date-range max: 2024-03-26


,No.,Event Time,User,URL,Policy Action,Cloud Application Class,Cloud Application,URL Class,URL Super Category,URL Category,...,ALPN Protocol,Unscannable Type,Forwarding Rule,Forwarding Method,Application Segment,Gateway IP,Gateway Name,Upload File Name,datetime,date
0,130,"March 22, 2024 9:00:04 PM PDT",AAAAA,d.la1-core1.sfdc-lywfpd.salesforceliveagent.co...,Allowed,Productivity and CRM Tools,Salesforce,Business Use,Business and Economy,Professional Services,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-22 14:00:04+00:00,2024-03-22
1,131,"March 22, 2024 9:00:04 PM PDT",AAAAA,d.la1-core1.sfdc-lywfpd.salesforceliveagent.co...,Allowed,Productivity and CRM Tools,Salesforce,Business Use,Business and Economy,Professional Services,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-22 14:00:04+00:00,2024-03-22
2,151,"March 22, 2024 9:00:05 PM PDT",AAAAA,chat.google.com:443,Allowed,Instant Messaging,Google Chat,Business Use,Internet Communication,Online Chat,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-22 14:00:05+00:00,2024-03-22
3,152,"March 22, 2024 9:00:05 PM PDT",AAAAA,chat.google.com/u/0/_/dynamitewebui/idv/,Allowed,Instant Messaging,Google Chat,Business Use,Internet Communication,Online Chat,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-22 14:00:05+00:00,2024-03-22
4,368,"March 22, 2024 9:00:12 PM PDT",AAAAA,d.la1-core2.sfdc-lywfpd.salesforceliveagent.co...,Allowed,Productivity and CRM Tools,Salesforce,Business Use,Business and Economy,Professional Services,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-22 14:00:12+00:00,2024-03-22


In [59]:
def count_by(col:str, df:pd.DataFrame)->pd.DataFrame:
    _df = df.groupby([col]).size().reset_index(name='count')
    _df.columns = [col, 'count']
    _df.sort_values(by='count', ascending=False, inplace=True)
    return _df

In [60]:
df_url = count_by('URL', df)
df_url.head()

,URL,count
112,client.wns.windows.com:443,74
111,client.wns.windows.com,68
205,landscape.canonical.com/ping,30
201,itam.assetsonar.com:443,23
281,s3.amazonaws.com,22


In [63]:
df_app = count_by('Cloud Application', df)
df_app.head()



,Cloud Application,count
9,Common Office 365 Applications,269
50,Slack,65
5,Amazon Web Services,57
16,DoubleClick,48
10,Criteo,48


In [ ]:
df_appclass = count_by('Cloud Application Class', df)
df_appclass.head()

In [ ]:
################################
# URL Access by user
#for each URL - count the number of access by each unique user.    
#  this allows multiple per users per day. 
df_url_by_user = df.groupby(['user', 'url']).size().reset_index(name='count')
df_url_by_user.columns = ['user', 'url', 'count']
df_url_by_user.sort_values(by='count', ascending=False, inplace=True)
df_url_by_user.head(10)